# ML / DL Prototype 

Time Estimate: 25 - 30 Hours / 50 - 60 Hours (depending on if you decide to build both prototypes)
The goal of this step is to find a machine learning or deep learning approach that works for your problem, and then show that the approach you choose is a viable one. Since the application has not been deployed to production yet, we’ll call it a prototype.

For this step, you’ll build your prototype in a Jupyter notebook. Depending on your problem, you’ll be using a more ‘traditional’ machine learning (ML) technique or a deep learning (DL) technique. Your goal is to come up with a working implementation of your prototype in a Jupyter notebook. This prototype could work on a subset of the data but demonstrates that your approach to solving the problem is a viable one based on the following criteria:
- The data has been reasonably split into training, validation, and test sets.
- You have used the correct metric(s) to evaluate the performance of your
algorithm.
- The performance of your algorithm is ‘good enough’ as determined by your mentor.

### Rubric

- The submission shows that the student understands how to design, implement, and justify a suitable machine learning and/or deep learning approach to a problem.
- The submission demonstrates that the student made thoughtful decisions about validating and testing the approach:
    - The data has been reasonably split into training, validation, and test sets
    - The ML/DL algorithm chosen for the approach makes sense in the context of the problem
    - The student has used the correct metric(s) to evaluate the performance of their algorithm
    - The performance of their algorithm is ‘good enough’ to meet the requirements of the business problem

- Well-documented GitHub repository and code. The Jupyter notebooks for the code provide step-by-step documentation that’s easy to follow. 

## ML Prototype

Summary: Light Gradient boosting optimized with SK Optimize

Pipeline
- feature extraction 
    - audio limit
    - melspectrogram (choose hyperparams)
        - fmin
        - fmax
        - n_mels (spectrogram height)
        - n_fft (window length in time)
        - hop_length
    - mfcc
        - hyperparams
    - chroma
    
- oversampling using ADASYN
- feature standardization using Quantile Transformer
- PCA (search components)
- LGBM
    - thorough hyperparameter search with skoptimize

## DL Prototype

Summary: 
1. CNN on melspectrogram followed by Linear Classifier
2. Create sequence of features and use LSTM